In [1]:
# import necessary packages
from typing import Sequence
import gradslam as gs
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from gradslam import Pointclouds, RGBDImages
from gradslam.datasets import TUM
from gradslam.slam import PointFusion
from torch.utils.data import DataLoader

In [2]:
path = "/scratch/saksham/data/tum"
sequences = "../sequences.txt"

In [3]:
# class TUM_D(torch.utils.data):
    
#     def __init__(self,dataset):
#         self.dataset=dataset
        
#     def __getitem__(self, index):
        
#         colors, depths, intrinsics, poses, *_  = self.dataset[index]
        





In [4]:
# load dataset
dataset = TUM(path, sequences = sequences , start = 56, seqlen = 30, height = 60, width = 80)
loader = DataLoader(dataset=dataset, batch_size=32)
colors, depths, intrinsics, poses, *_ = next(iter(loader))

In [5]:
# create rgbdimages object
rgbdimages = RGBDImages(colors, depths, intrinsics, poses)

In [6]:
colors, depths, intrinsics, poses, *_ = dataset[0]

In [7]:
# step by step SLAM
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
slam = PointFusion(device=device)

In [8]:
pointclouds = Pointclouds(device=device)
pointclouds, recovered_poses = slam(rgbdimages)

In [9]:
pts3D = (pointclouds.points_list[0]).cpu().numpy()

In [10]:
poses = poses.numpy()

In [11]:
poses = poses[:, :3]

In [12]:
bounds = np.zeros((len(poses), 2))
visibilities = np.zeros((len(poses), len(pts3D)))
pts_world = np.zeros((1, 3, len(pts3D))) # (1, 3, N_points)
visibilities = np.zeros((len(poses), len(pts3D))) # (N_images, N_points)

In [13]:
for i, k in enumerate(pts3D):
    pts_world[0, :, i] = pts3D[i]

In [14]:
depths = ((pts_world-poses[..., 3:4])*poses[..., 2:3]).sum(1) 

In [15]:
for i in range(len(poses)):
    zs = depths[i]
    bounds[i] = [np.percentile(zs, 0.1), np.percentile(zs, 99.9)]

In [16]:
near = bounds.min()

In [17]:
far = bounds.max()

In [18]:
near

0.6634061115078476

In [19]:
far

3.5994947452704773

In [20]:
scale_factor = far/0.75

In [21]:
scale_factor

4.799326327027303